<a href="https://colab.research.google.com/github/ShyamSundhar1411/My-ML-Notebooks/blob/master/Natural%20Language%20Processing/Introduction_to_NLP_in_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction to NLP Fundamentals in Tensorflow

In [91]:
!nvidia-smi

Tue Feb 21 18:32:06 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P0    27W /  70W |   5084MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [92]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2023-02-21 18:32:06--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py.1’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-02-21 18:32:06 (100 MB/s) - ‘helper_functions.py.1’ saved [10246/10246]



In [93]:
from helper_functions import plot_loss_curves,unzip_data,create_tensorboard_callback,compare_historys

## Getting a text Dataset

In [94]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

--2023-02-21 18:32:06--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.202.128, 74.125.69.128, 64.233.182.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.202.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip.1’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.004s  

2023-02-21 18:32:06 (147 MB/s) - ‘nlp_getting_started.zip.1’ saved [607343/607343]



In [95]:
unzip_data("nlp_getting_started.zip")

## Visualizing Dataset

In [96]:
import pandas as pd
train_dataset = pd.read_csv("/content/train.csv")

In [97]:
train_dataset

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [98]:
train_dataset_shuffled = train_dataset.sample(frac = 1,random_state = 42)


In [99]:
train_dataset_shuffled

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0
...,...,...,...,...,...
5226,7470,obliteration,Merica!,@Eganator2000 There aren't many Obliteration s...,0
5390,7691,panic,NaN,just had a panic attack bc I don't have enough...,0
860,1242,blood,NaN,Omron HEM-712C Automatic Blood Pressure Monito...,0
7603,10862,NaN,NaN,Officials say a quarantine is in place at an A...,1


In [100]:
test_data  = pd.read_csv("/content/test.csv")

In [101]:
test_data

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [102]:
train_dataset.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [103]:
import random
random_index = random.randint(0,len(train_dataset)-5)
for row in train_dataset_shuffled[["text","target"]][random_index:random_index+5].itertuples():
  _,text,target = row
  print(text,target)

@MarianKeyes Rubber Mudslide! Still laughing! 0
It's cold and my head wants to explode.. The joys of working from home - I'm going back to bed / peace out ???? 0
Governor allows parole for California school bus hijacker who kidnapped 26 children in 1976. http://t.co/hdAhLgrprl http://t.co/Z1s3T77P3L 1
@DavisKawalya I know @Mauryn143 will be saying her final goodbyes to grandpa as seen on news RiP Me? always open to ideas but may ve curfew 0
China is only delaying the deluge: If the fundamentals of an economy do not support the valuations of a stock ... http://t.co/fwIkyUrC18 0


## Split Data into Training and Validation Sets

In [104]:
from sklearn.model_selection import train_test_split

In [105]:
train_sentences,val_sentences,train_labels,val_labels = train_test_split(
    train_dataset_shuffled['text'].to_numpy(),
    train_dataset_shuffled['target'].to_numpy(),
    test_size = 0.1,
    random_state = 42
    )

In [106]:
train_sentences[:10]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk',
       '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
       'destroy the free fandom honestly',
       'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
       '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
       'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
      dtype=object)

## Tokenization and Embedding

### Text Vectorization (Tokenization)

In [107]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

text_vectorizer = TextVectorization(
    max_tokens = 10000,
    standardize = "lower_and_strip_punctuation",
    split = "whitespace",
    ngrams = None,
    output_mode = "int",
    output_sequence_length = None,
    pad_to_max_tokens = True,
  )


In [108]:
round(sum([len(i.split()) for i in train_sentences]))/len(train_sentences)

14.901036345059115

### Text Vecotrizer Variables

In [109]:
max_vocab_length = 10000
max_length = 15
text_vectorizer = TextVectorization(
    max_tokens = max_vocab_length,
    standardize = "lower_and_strip_punctuation",
    split = "whitespace",
    ngrams = None,
    output_mode = "int",
    output_sequence_length = max_length,
    pad_to_max_tokens = True,
)

In [110]:
text_vectorizer.adapt(train_sentences)

In [111]:
sample_sentence = "Flood in my street"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [112]:
random_sentences = random.choice(train_sentences)
text_vectorizer([random_sentences])


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 966, 1789,  662,    7, 3513,    4,    2,  274,  894, 1138,  504,
          18, 1691, 6786,    1]])>

In [113]:
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
top_5_words

['', '[UNK]', 'the', 'a', 'in']

## Embedding Layer

In [114]:
from tensorflow.keras import layers
embedding = layers.Embedding(input_dim = max_vocab_length,
      output_dim = 128,
      input_length = max_length
                                   
)
embedding

In [115]:
random_sentence = random.choice(train_sentences)
print(random_sentence)
embedded_sentence = embedding(text_vectorizer(random_sentence))
print(embedded_sentence)

Turning rubble from disasters into 'Lego' bricks you can build houses with. This one belongs in #crazyideascollege 
http://t.co/dh0s4bUuK7
tf.Tensor(
[[ 0.02915055  0.03454695 -0.01336664 ... -0.03364633  0.00981926
   0.01973389]
 [-0.04841442  0.03640108 -0.01884978 ... -0.04007479 -0.0185073
  -0.04917897]
 [ 0.02763948 -0.01263096 -0.01369139 ... -0.04223433  0.04526644
  -0.00611509]
 ...
 [ 0.04448823  0.04971189  0.01929443 ...  0.01343041 -0.03816766
  -0.04314127]
 [-0.00791184 -0.0039448  -0.02948331 ... -0.03962351 -0.00644801
  -0.0429091 ]
 [ 0.02191234  0.00223626 -0.0152855  ... -0.03137027  0.00263993
  -0.00897555]], shape=(15, 128), dtype=float32)


## Model 0 : Naive Bayes (Baseline)

In [116]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_0 = Pipeline([
    ("tfidf",TfidfVectorizer()), #convert words to numbers,
    ("clf",MultinomialNB()), #model the text,
])
model_0.fit(train_sentences,train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [117]:
base_line_score = model_0.score(val_sentences,val_labels)

In [118]:
base_line_score

0.7926509186351706

In [119]:
base_line_predict = model_0.predict(val_sentences)
base_line_predict[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

## Creating functions to compare model experiments

In [120]:
from sklearn.metrics import accuracy_score,precision_recall_fscore_support

def calculate_results(y_true,y_pred):
  model_accuracy = accuracy_score(y_true,y_pred)*100
  model_precision,model_recall,model_f1,_ = precision_recall_fscore_support(y_true,y_pred,average = "weighted")
  model_results = {
      "accuracy":model_accuracy,
      "precision":model_precision,
      "recall":model_recall,
      "f1":model_f1
  }
  return model_results

In [121]:
base_line_results = calculate_results(val_labels,base_line_predict)
base_line_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

## Model 1: Simple Dense Model

In [122]:
from helper_functions import create_tensorboard_callback
SAVE_DIR = "model_logs"

In [123]:
from tensorflow.keras import layers

inputs = layers.Input(shape = (1,),dtype = tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GlobalMaxPool1D()(x)
outputs = layers.Dense(1,activation = "sigmoid")(x)
model_1 = tf.keras.Model(inputs,outputs,name = "model_1_dense")

In [124]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_3 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 global_max_pooling1d_3 (Glo  (None, 128)              0         
 balMaxPooling1D)                                                
                                                                 
 dense_7 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [125]:
model_1.compile(loss = "binary_crossentropy",
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ['accuracy'])

In [126]:
model_1_history = model_1.fit(
    x = train_sentences,y = train_labels,
    epochs = 5,
    validation_data = (val_sentences,val_labels),
    callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR,experiment_name = "model_1_dense")]
    )

Saving TensorBoard log files to: model_logs/model_1_dense/20230221-183208
Epoch 1/5
215/215 [==============================] - 9s 38ms/step - loss: 0.6382 - accuracy: 0.6533 - val_loss: 0.5732 - val_accuracy: 0.7690
Epoch 2/5
215/215 [==============================] - 2s 11ms/step - loss: 0.4683 - accuracy: 0.8311 - val_loss: 0.4721 - val_accuracy: 0.7848
Epoch 3/5
215/215 [==============================] - 3s 12ms/step - loss: 0.3397 - accuracy: 0.8758 - val_loss: 0.4549 - val_accuracy: 0.7822
Epoch 4/5
215/215 [==============================] - 2s 12ms/step - loss: 0.2566 - accuracy: 0.9078 - val_loss: 0.4543 - val_accuracy: 0.7874
Epoch 5/5
215/215 [==============================] - 4s 18ms/step - loss: 0.1969 - accuracy: 0.9340 - val_loss: 0.4673 - val_accuracy: 0.7913


In [127]:
model_1.evaluate(val_sentences,val_labels)

24/24 [==============================] - 0s 6ms/step - loss: 0.4673 - accuracy: 0.7913


[0.46734338998794556, 0.7913385629653931]

In [128]:
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs.shape

24/24 [==============================] - 0s 4ms/step


(762, 1)

In [129]:
model_1_pred_probs[10]

array([0.20301238], dtype=float32)

In [130]:
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))

In [131]:
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)>

In [132]:
model_1_results = calculate_results(y_true = val_labels,y_pred = model_1_preds)
model_1_results

{'accuracy': 79.13385826771653,
 'precision': 0.7949959084822513,
 'recall': 0.7913385826771654,
 'f1': 0.7888851650587367}

## Visualizing Learned Embeddings

In [133]:
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab)

10000

In [134]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_3 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 global_max_pooling1d_3 (Glo  (None, 128)              0         
 balMaxPooling1D)                                                
                                                                 
 dense_7 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [136]:
embed_weights = model_1.get_layer("embedding_1").get_weights()[0]

In [137]:
embed_weights.shape

(10000, 128)

In [138]:
embed_weights

array([[-0.03501402, -0.11233395, -0.10881624, ..., -0.01484641,
        -0.10534103, -0.04709535],
       [-0.03436721, -0.04529749, -0.0610215 , ..., -0.02029046,
        -0.10953031, -0.00791831],
       [-0.03959833, -0.04694129, -0.05909704, ..., -0.02340883,
        -0.07763607, -0.02036991],
       ...,
       [-0.01492535,  0.02769333,  0.04147359, ..., -0.03174561,
         0.03473841,  0.0471264 ],
       [ 0.07981313, -0.02653029, -0.04209688, ...,  0.02487648,
        -0.04083954,  0.01987699],
       [-0.03169294, -0.00201584, -0.0098873 , ..., -0.01634334,
         0.00200887, -0.04515908]], dtype=float32)

## Recurrent Neural Network

### Model 2: LSTM

In [139]:
from tensorflow.keras import layers
inputs = layers.Input(shape = (1,),dtype = "string")
x = text_vectorizer(inputs)
x = embedding(x)
print(x.shape)
x = layers.LSTM(64,return_sequences = True)(x) # When we are stacking we need return_sequences = True
print(x.shape)
x = layers.LSTM(64)(x)
print(x.shape)
x = layers.Dense(64,activation = "relu")(x)
outputs = layers.Dense(1,activation = "sigmoid")(x)
model_2 = tf.keras.Model(inputs,outputs,name = "model_2_LSTM")

(None, 15, 128)
(None, 15, 64)
(None, 64)


In [140]:
model_2.compile(loss = "binary_crossentropy",optimizer = "adam",metrics = ['accuracy'])

In [141]:
model_2_history = model_2.fit(train_sentences,train_labels,epochs = 5,
            validation_data = (val_sentences,val_labels),
            callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR,experiment_name = "model_2_lstm")]
            )

Saving TensorBoard log files to: model_logs/model_2_lstm/20230221-183241
Epoch 1/5
215/215 [==============================] - 22s 77ms/step - loss: 0.3683 - accuracy: 0.8437 - val_loss: 0.4721 - val_accuracy: 0.7940
Epoch 2/5
215/215 [==============================] - 6s 27ms/step - loss: 0.2400 - accuracy: 0.9072 - val_loss: 0.5066 - val_accuracy: 0.7900
Epoch 3/5
215/215 [==============================] - 5s 24ms/step - loss: 0.1788 - accuracy: 0.9321 - val_loss: 0.6082 - val_accuracy: 0.7795
Epoch 4/5
215/215 [==============================] - 5s 23ms/step - loss: 0.1314 - accuracy: 0.9512 - val_loss: 0.7550 - val_accuracy: 0.7664
Epoch 5/5
215/215 [==============================] - 4s 19ms/step - loss: 0.0911 - accuracy: 0.9628 - val_loss: 0.7486 - val_accuracy: 0.7717


In [142]:
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]

24/24 [==============================] - 1s 5ms/step


array([[0.11390454],
       [0.6998338 ],
       [0.9986222 ],
       [0.39161092],
       [0.00217489],
       [0.9995135 ],
       [0.17287749],
       [0.99991906],
       [0.9999254 ],
       [0.4466527 ]], dtype=float32)

In [143]:
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))

In [144]:
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 0., 1., 1., 0.], dtype=float32)>

In [145]:
calculate_results(val_labels,model_2_preds)

{'accuracy': 77.16535433070865,
 'precision': 0.7747861668850706,
 'recall': 0.7716535433070866,
 'f1': 0.7688960790251899}

### Model-3 GRU Powered RNN

In [146]:
from tensorflow.keras import layers
inputs = layers.Input(shape = (1,),dtype = "string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GRU(64)(x)
#If returns sequences and stackings are true
# x = layers.LSTM(64,return_sequences = True)(x)
# x = layers.GRU(64)(x)
# x = layers.Dense(64,activation = "relu")(x)
# x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1,activation = "sigmoid")(x)
model_3 = tf.keras.Model(inputs,outputs,name = "model_3_GRU")

In [147]:
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_3 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 gru_1 (GRU)                 (None, 64)                37248     
                                                                 
 dense_10 (Dense)            (None, 1)                 65        
                                                                 
Total params: 1,317,313
Trainable params: 1,317,313
Non-trainable params: 0
_____________________________________________

In [148]:
model_3.compile(loss = "binary_crossentropy",metrics = ['accuracy'],optimizer = "adam")

In [149]:
model_3_history = model_3.fit(train_sentences,train_labels,epochs = 5,
            validation_data = (val_sentences,val_labels),
            callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR,experiment_name = "model_3_gru")]
            )

Saving TensorBoard log files to: model_logs/model_3_gru/20230221-183328
Epoch 1/5
215/215 [==============================] - 11s 40ms/step - loss: 0.2233 - accuracy: 0.9000 - val_loss: 0.7424 - val_accuracy: 0.7743
Epoch 2/5
215/215 [==============================] - 3s 13ms/step - loss: 0.1026 - accuracy: 0.9647 - val_loss: 0.7124 - val_accuracy: 0.7717
Epoch 3/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0769 - accuracy: 0.9710 - val_loss: 0.8474 - val_accuracy: 0.7651
Epoch 4/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0556 - accuracy: 0.9756 - val_loss: 1.1706 - val_accuracy: 0.7612
Epoch 5/5
215/215 [==============================] - 2s 7ms/step - loss: 0.0523 - accuracy: 0.9769 - val_loss: 1.3048 - val_accuracy: 0.7664


In [150]:
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs[:10]

24/24 [==============================] - 0s 3ms/step


array([[8.4750802e-04],
       [7.2912347e-01],
       [9.9978071e-01],
       [1.6065556e-01],
       [1.4704987e-04],
       [9.9983609e-01],
       [6.4169526e-01],
       [9.9992919e-01],
       [9.9992931e-01],
       [2.9303437e-01]], dtype=float32)

In [151]:
model_3_pred = tf.squeeze(tf.round(model_3_pred_probs))

In [152]:
model_3_pred[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [153]:
calculate_results(model_3_pred,val_labels)

{'accuracy': 76.64041994750657,
 'precision': 0.7811822435239074,
 'recall': 0.7664041994750657,
 'f1': 0.769376278201219}

### Model-4 Bidirectional RNN

In [154]:
from tensorflow.keras import layers
inputs = layers.Input(shape = (1,),dtype = tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Bidirectional(layers.LSTM(64,return_sequences = True))(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
outputs = layers.Dense(1,activation = "sigmoid",)(x)
model_4 = tf.keras.Model(inputs,outputs)

In [155]:
model_4.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_3 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 bidirectional_2 (Bidirectio  (None, 15, 128)          98816     
 nal)                                                            
                                                                 
 bidirectional_3 (Bidirectio  (None, 128)              98816     
 nal)                                                            
                                                           

In [156]:
model_4.compile(loss = "binary_crossentropy",optimizer = "adam",metrics = ['accuracy'])

In [157]:
mode_4_history = model_4.fit(
    train_sentences,train_labels,epochs = 5,validation_data = (val_sentences,val_labels),
    callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR,experiment_name = "model_4_bidirectional")],
)

Saving TensorBoard log files to: model_logs/model_4_bidirectional/20230221-183350
Epoch 1/5
215/215 [==============================] - 22s 65ms/step - loss: 0.1299 - accuracy: 0.9505 - val_loss: 0.9364 - val_accuracy: 0.7572
Epoch 2/5
215/215 [==============================] - 9s 41ms/step - loss: 0.0616 - accuracy: 0.9729 - val_loss: 1.2471 - val_accuracy: 0.7507
Epoch 3/5
215/215 [==============================] - 3s 16ms/step - loss: 0.0565 - accuracy: 0.9746 - val_loss: 1.2525 - val_accuracy: 0.7625
Epoch 4/5
215/215 [==============================] - 3s 14ms/step - loss: 0.0496 - accuracy: 0.9778 - val_loss: 1.2398 - val_accuracy: 0.7559
Epoch 5/5
215/215 [==============================] - 4s 17ms/step - loss: 0.0735 - accuracy: 0.9746 - val_loss: 1.2076 - val_accuracy: 0.7612


In [158]:
model_4_preds = tf.squeeze(tf.round(model_4.predict(val_sentences)))

24/24 [==============================] - 1s 5ms/step


In [159]:
calculate_results(val_labels,model_4_preds)

{'accuracy': 76.11548556430446,
 'precision': 0.7639062094795714,
 'recall': 0.7611548556430446,
 'f1': 0.758270611394164}

### Model-5: Conv1D 

In [160]:
embedding_text = embedding(text_vectorizer(['this is a test sentence for embedding']))

In [161]:
embedding_text

<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.06946975, -0.06262796, -0.04778665, ..., -0.03422462,
         -0.06615641, -0.04656116],
        [-0.02238939, -0.05452709, -0.01980302, ..., -0.06210402,
         -0.03210992,  0.08788482],
        [ 0.02660478, -0.08194113, -0.035379  , ..., -0.01539939,
         -0.02191291, -0.03268178],
        ...,
        [-0.04197207, -0.04551236, -0.04969615, ..., -0.0264189 ,
         -0.06391979, -0.06163619],
        [-0.04197207, -0.04551236, -0.04969615, ..., -0.0264189 ,
         -0.06391979, -0.06163619],
        [-0.04197207, -0.04551236, -0.04969615, ..., -0.0264189 ,
         -0.06391979, -0.06163619]]], dtype=float32)>

In [162]:
conv_1d = layers.Conv1D(filters = 32,
kernel_size = 5,
activation = "relu",
padding = "same"
)

In [163]:
conv_1d_output = conv_1d(embedding_text)
max_pool = layers.GlobalMaxPool1D()
max_pool_output = max_pool(conv_1d_output)
embedding_text.shape, conv_1d_output.shape,max_pool_output.shape

(TensorShape([1, 15, 128]), TensorShape([1, 15, 32]), TensorShape([1, 32]))

In [164]:
max_pool_output

<tf.Tensor: shape=(1, 32), dtype=float32, numpy=
array([[0.16488244, 0.        , 0.11560898, 0.10295957, 0.04306193,
        0.09378796, 0.        , 0.10126154, 0.04976948, 0.09445468,
        0.10385998, 0.09373024, 0.14394212, 0.        , 0.02179516,
        0.06521705, 0.15576333, 0.09300973, 0.06833167, 0.06126281,
        0.03078514, 0.14418921, 0.0301302 , 0.13137326, 0.06711338,
        0.        , 0.11524815, 0.00249872, 0.06362821, 0.03421468,
        0.        , 0.10576242]], dtype=float32)>

In [165]:
from tensorflow.keras import layers
inputs = layers.Input(shape = (1,),dtype = tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Conv1D(filters = 64,kernel_size = 5,activation = "relu",padding = "valid")(x)
x = layers.GlobalMaxPool1D()(x)
outputs = layers.Dense(1,activation = "sigmoid")(x)
model_5 = tf.keras.Model(inputs,outputs,name = "model_5_conv1d")

In [166]:
model_5.compile(loss = "binary_crossentropy",optimizer = "adam",metrics = ['accuracy'])

In [167]:
mode_5_history = model_5.fit(
    train_sentences,train_labels,epochs = 5,validation_data = (val_sentences,val_labels),
    callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR,experiment_name = "model_5_conv")],
)

Saving TensorBoard log files to: model_logs/model_5_conv/20230221-183433
Epoch 1/5
215/215 [==============================] - 9s 39ms/step - loss: 0.1455 - accuracy: 0.9593 - val_loss: 0.8810 - val_accuracy: 0.7572
Epoch 2/5
215/215 [==============================] - 3s 12ms/step - loss: 0.0725 - accuracy: 0.9753 - val_loss: 1.0266 - val_accuracy: 0.7664
Epoch 3/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0601 - accuracy: 0.9793 - val_loss: 1.0855 - val_accuracy: 0.7598
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0534 - accuracy: 0.9800 - val_loss: 1.1420 - val_accuracy: 0.7585
Epoch 5/5
215/215 [==============================] - 2s 7ms/step - loss: 0.0499 - accuracy: 0.9809 - val_loss: 1.1461 - val_accuracy: 0.7546


In [168]:
model_5_pred_probs = model_5.predict(val_sentences)
model_5_preds = tf.squeeze(tf.round(model_5_pred_probs))

24/24 [==============================] - 0s 2ms/step


In [169]:
calculate_results(val_labels,model_5_preds)

{'accuracy': 75.45931758530183,
 'precision': 0.7542851958949438,
 'recall': 0.7545931758530183,
 'f1': 0.7543752032864434}

### Model 6: TFHub Pretrained Sentence Encoder

In [170]:
import tensorflow_hub as hub

In [171]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

In [172]:
samples = embed([sample_sentence,"When you call universal sentence , turns into numbers"])

In [173]:
print(samples)

tf.Tensor(
[[-0.00165963  0.00170079  0.04788961 ...  0.00854664  0.00299551
  -0.04015192]
 [ 0.0568927  -0.08954596 -0.0283566  ... -0.00659815  0.05909768
  -0.01464346]], shape=(2, 512), dtype=float32)


In [174]:
sentence_encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
            input_shape = [],
            dtype = tf.string,
            trainable = False,
            name = "USE"
        )

In [175]:
model_6 = tf.keras.Sequential([
    sentence_encoder_layer,
    layers.Dense(1,activation = "sigmoid"),
],name = "model_6_USE")

In [176]:
model_6.compile(loss = "binary_crossentropy",optimizer = "adam",metrics = ["accuracy"])

In [177]:
mode_6_history = model_6.fit(
    train_sentences,train_labels,epochs = 5,validation_data = (val_sentences,val_labels),
    callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR,experiment_name = "model_6_use")],
)

Saving TensorBoard log files to: model_logs/model_6_use/20230221-183505
Epoch 1/5
215/215 [==============================] - 11s 29ms/step - loss: 0.6471 - accuracy: 0.7341 - val_loss: 0.6123 - val_accuracy: 0.7769
Epoch 2/5
215/215 [==============================] - 5s 25ms/step - loss: 0.5809 - accuracy: 0.7914 - val_loss: 0.5639 - val_accuracy: 0.7887
Epoch 3/5
215/215 [==============================] - 4s 18ms/step - loss: 0.5383 - accuracy: 0.7952 - val_loss: 0.5323 - val_accuracy: 0.7848
Epoch 4/5
215/215 [==============================] - 4s 17ms/step - loss: 0.5097 - accuracy: 0.7977 - val_loss: 0.5114 - val_accuracy: 0.7848
Epoch 5/5
215/215 [==============================] - 6s 27ms/step - loss: 0.4899 - accuracy: 0.7996 - val_loss: 0.4971 - val_accuracy: 0.7861


In [186]:
model_6_pred_probs = model_6.predict(val_sentences)
model_6_pred_labels = tf.squeeze(tf.round(model_6_pred_probs))

24/24 [==============================] - 1s 24ms/step


In [179]:
calculate_results(val_labels,model_6_pred_labels)

{'accuracy': 78.60892388451444,
 'precision': 0.7862161422078583,
 'recall': 0.7860892388451444,
 'f1': 0.7851501923128148}

### Model-7 : 10% of the Data

In [183]:
train_10_percent = train_dataset[['text','target']].sample(frac = 0.1,random_state = 42)

In [184]:
train_sentences_10_percent = train_10_percent['text'].to_list()
train_labels_10_percent = train_10_percent['target'].to_list()

In [185]:
train_10_percent[:10]

,text,target
2644,So you have a new weapon that can cause un-ima...,1
2227,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,Aftershock back to school kick off was great. ...,0
6845,in response to trauma Children of Addicts deve...,0
5559,@Calum5SOS you look like you got caught in a r...,0
1765,my favorite lady came to our volunteer meeting...,1
1817,@brianroemmele UX fail of EMV - people want to...,1
6810,Can't find my ariana grande shirt this is a f...,0
4398,The Murderous Story Of AmericaÛªs First Hijac...,1


In [187]:
train_10_percent['target'].value_counts()

0    425
1    336
Name: target, dtype: int64

In [190]:
model_7 = tf.keras.Sequential([
    sentence_encoder_layer,
    tf.keras.layers.Dense(100,activation = "relu"),
    tf.keras.layers.Dense(1,activation = "sigmoid"),
])
model_7.compile(loss = "binary_crossentropy",optimizer = "adam",metrics = ['accuracy'])

In [191]:
model_7.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_18 (Dense)            (None, 100)               51300     
                                                                 
 dense_19 (Dense)            (None, 1)                 101       
                                                                 
Total params: 256,849,225
Trainable params: 51,401
Non-trainable params: 256,797,824
_________________________________________________________________


In [192]:
mode_7_history = model_7.fit(
    train_sentences,train_labels,epochs = 5,validation_data = (val_sentences,val_labels),
    callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR,experiment_name = "model_7_use")],
)

Saving TensorBoard log files to: model_logs/model_7_use/20230221-184028
Epoch 1/5
215/215 [==============================] - 6s 15ms/step - loss: 0.4973 - accuracy: 0.7797 - val_loss: 0.4448 - val_accuracy: 0.8031
Epoch 2/5
215/215 [==============================] - 3s 13ms/step - loss: 0.4123 - accuracy: 0.8165 - val_loss: 0.4363 - val_accuracy: 0.8110
Epoch 3/5
215/215 [==============================] - 4s 17ms/step - loss: 0.3982 - accuracy: 0.8243 - val_loss: 0.4271 - val_accuracy: 0.8176
Epoch 4/5
215/215 [==============================] - 3s 13ms/step - loss: 0.3890 - accuracy: 0.8270 - val_loss: 0.4262 - val_accuracy: 0.8163
Epoch 5/5
215/215 [==============================] - 4s 16ms/step - loss: 0.3786 - accuracy: 0.8345 - val_loss: 0.4230 - val_accuracy: 0.8215


In [193]:
model_7_pred_probs = model_7.predict(val_sentences)

24/24 [==============================] - 1s 17ms/step


In [194]:
model_7_pred = tf.squeeze(tf.round(model_7_pred_probs))

In [195]:
calculate_results(val_labels,model_7_pred)

{'accuracy': 82.1522309711286,
 'precision': 0.8225211063579874,
 'recall': 0.821522309711286,
 'f1': 0.8205421576435756}